# Exploración de los datos de entrenamiento

Autores:
- Pablo Quito
- Juan Valdiviezo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('spanish'))

In [ ]:
#Inicialización
ROUTE = '/content/drive/My Drive/Intelektubies/Datos/Entrenamiento V5'
FILE_NAME = 'df_entrenamiento_v5.xlsx'#Ultima versión de de entrenamiento
DATASET_PATH = ROUTE+'/'+FILE_NAME
drive.mount('/content/drive')
df = pd.read_excel(DATASET_PATH)
print(df.shape)
df.head()

In [ ]:
#Verificar nombres de columnas y nulos
print(f"Columnas: {df.columns.values} ")
print(df.isnull().sum())
df.dropna(inplace=True) #Eliminar elementos nulos

In [ ]:
df.dtypes

In [ ]:
print(f"Rating Sentimiento: \n{df['sentimiento'].value_counts()}")

In [ ]:
counts = df['sentimiento'].value_counts()
bars = counts.plot.bar(color='#002856')

plt.title('Distribución de Comentarios por Tipo de Sentimiento', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Tipo de Sentimiento', fontsize=11,labelpad=20)

# Eliminar eje Y completamente
plt.gca().axes.get_yaxis().set_visible(False)

# Etiquetas sobre cada barra
for bar in bars.patches:
    plt.text(bar.get_x() + bar.get_width()/2.0, bar.get_height() + 0.1, int(bar.get_height()),
             ha='center', va='bottom', fontsize=12)

# Ajustes de etiquetas del eje X
plt.xticks(rotation=0, fontsize=11)

# Eliminar el marco
for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
print(f"Porcentaje - Distribución: \n{round(df['sentimiento'].value_counts()/df.shape[0]*100,2)}")


In [ ]:
import matplotlib.pyplot as plt
from io import BytesIO

fig = plt.figure(figsize=(7, 7))

# Paleta armónica basada en #002856
colors = ['#003656', '#005377', '#3399cc', '#66c2ff']

wp = {'linewidth': 1, "edgecolor": 'white'}

tags = df['sentimiento'].value_counts() / df.shape[0]
explode = (0.05,) * len(tags)  # Uniforme según número de clases

# Crear el gráfico manualmente para controlar etiquetas
ax = fig.add_subplot(111)
wedges, texts, autotexts = ax.pie(
    tags,
    labels=tags.index,
    autopct='%1.1f%%',
    shadow=False,
    colors=colors[:len(tags)],
    startangle=90,
    wedgeprops=wp,
    explode=explode,
    labeldistance=1.05
)

# Modificar estilo de etiquetas (categorías)
for text in texts:
    text.set_fontsize(12)         # Tamaño de fuente de las categorías
    text.set_fontweight('bold')   # Negrita

# Modificar estilo de los porcentajes
for autotext in autotexts:
    autotext.set_fontsize(11)     # Tamaño de porcentaje
    autotext.set_color('white')   # Opcional: blanco sobre colores oscuros
    autotext.set_fontweight('bold')

# Título y eje
ax.set_title('Distribución Proporcional de Comentarios por Sentimiento', fontsize=14, fontweight='bold', pad=20)
ax.set_ylabel('')

# Guardar imagen
graph = BytesIO()
fig.savefig(graph, format="png", dpi=300)
graph.seek(0)


In [ ]:
#Distribución de los comentarios
sns.histplot(df['comentario'].apply(len),color='blue').set(title='Distribución de caracteres de comentarios ')

In [ ]:
data = df.copy()
data['length'] = df['comentario'].apply(len)
print(df)
data

In [ ]:
# Corregir el filtro: filtramos por la columna 'sentimiento'
sns.histplot(data[data['sentimiento'] == 'Positiva']['length'], color='green')\
   .set(title='Distribución de longitud de comentarios Positivos')

# Mostrar el gráfico
plt.show()


In [ ]:
# Corregir el filtro: filtramos por la columna 'sentimiento'
sns.histplot(data[data['sentimiento'] == 'Negativa']['length'], color='red')\
   .set(title='Distribución de longitud de comentarios Negativos')

plt.show()


In [ ]:
sns.histplot(data[data['sentimiento'] == 'Alerta']['length'], color='black')\
   .set(title='Distribución de longitud de comentarios Alerta')

plt.show()

In [ ]:
sns.histplot(data[data['sentimiento'] == 'Neutral']['length'], color='orange')\
   .set(title='Distribución de longitud de comentarios Neutral')

plt.show()

In [ ]:
data['sentimiento_num'] = data['sentimiento'].map({'Positiva': 2, 'Negativa': 0, 'Neutral': 1,'Alerta': 3}).astype(int)
data.groupby('length')['sentimiento_num'].mean().plot.hist(color = 'blue', bins = 20)
plt.title(" Review length wise mean sentiment")
plt.xlabel('sentimiento')
plt.ylabel('length')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Obtener las stopwords en español y convertirlas en una lista
spanish_stopwords = list(stopwords.words('spanish'))


In [ ]:
# Añadir las palabras personalizadas a las stopwords
custom_stopwords = ["debe","mejorar", "destaca","estudiante","estudiantes","docente",
                    "clase","clases","materia","materias","tema","forma","manera","temas",
                    "ser","debería","solo","hace"]
spanish_stopwords.extend(custom_stopwords)  # Usar extend para añadir las palabras

# Crear el CountVectorizer con las stopwords en español
cv = CountVectorizer(stop_words=spanish_stopwords)

# Ajustar y transformar los datos
words = cv.fit_transform(data.comentario)

In [ ]:
from wordcloud import WordCloud
# Combine all reviews
reviews = " ".join([review for review in data['comentario']])

# Initialize wordcloud object
wc = WordCloud(background_color='white', max_words=50, stopwords=spanish_stopwords)

# Generate and plot wordcloud
plt.figure(figsize=(10,10))
plt.imshow(wc.generate(reviews))
plt.title('Wordcloud for all reviews', fontsize=10)
plt.axis('off')
plt.show()

In [ ]:
neg_comentarios = " ".join([review for review in data[data['sentimiento'] == 'Negativa']['comentario']])
neg_comentarios = neg_comentarios.lower()
pos_comentarios = " ".join([review for review in data[data['sentimiento'] == 'Positiva']['comentario']])
pos_comentarios = pos_comentarios.lower()
neu_comentarios = " ".join([review for review in data[data['sentimiento'] == 'Neutral']['comentario']])
neu_comentarios = neu_comentarios.lower()
alert_comentarios = " ".join([review for review in data[data['sentimiento'] == 'Alerta']['comentario']])
alert_comentarios = alert_comentarios.lower()


In [ ]:
wc = WordCloud(background_color='white', max_words=50, stopwords=spanish_stopwords)

# Generate and plot wordcloud
plt.figure(figsize=(10,10))
plt.imshow(wc.generate(neg_comentarios))
plt.title('Wordcloud for negative reviews', fontsize=10)
plt.axis('off')
plt.show()

In [ ]:
wc = WordCloud(background_color='white', max_words=50, stopwords=spanish_stopwords)

# Generate and plot wordcloud
plt.figure(figsize=(10,10))
plt.imshow(wc.generate(pos_comentarios))
plt.title('Wordcloud for positive reviews', fontsize=10)
plt.axis('off')
plt.show()

In [ ]:
wc = WordCloud(background_color='white', max_words=50, stopwords=spanish_stopwords)

# Generate and plot wordcloud
plt.figure(figsize=(10,10))
plt.imshow(wc.generate(neu_comentarios))
plt.title('Wordcloud for neutral reviews', fontsize=10)
plt.axis('off')
plt.show()

In [ ]:
wc = WordCloud(background_color='white', max_words=50, stopwords=spanish_stopwords)

# Generate and plot wordcloud
plt.figure(figsize=(10,10))
plt.imshow(wc.generate(alert_comentarios))
plt.title('Wordcloud for alert reviews', fontsize=10)
plt.axis('off')
plt.show()